# Imports

In [2]:
# General imports.
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import os

# ML-specific imports.
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import evaluate
import wandb

2023-12-15 12:02:46.311903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Configs

In [2]:
wandb.init(project="ArguMentor")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rafael_piacsek. Use `wandb login --relogin` to force relogin


In [3]:
id2label = {
  0:  'Unnanotated',
  1:  'B-Lead',
  2:  'I-Lead',
  3:  'B-Position',
  4:  'I-Position',
  5:  'B-Evidence',
  6:  'I-Evidence',
  7:  'B-Claim',
  8:  'I-Claim',
  9:  'B-Concluding_Statement',
  10: 'I-Concluding_Statement',
  11: 'B-Counterclaim',
  12: 'I-Counterclaim',
  13: 'B-Rebuttal',
  14: 'I-Rebuttal'
}

label2id = {v: k for k, v in id2label.items()}

In [4]:
config = {
    'model_name': 'google/bigbird-roberta-base',
    'output_dir': './model_test/',
    'validation_split': .15,
    'seed': 3,
    'max_length': 1024,
    'device': 'cuda' if torch.cuda.is_available()
                        else 'mps'
                     if torch.backends.mps.is_available()
                        else 'cpu' # Added support for Apple Metal acceleration.
}

run_name = f"{config['model_name'].replace('/', '-')}-{time.strftime('%Y-%m-%d_%H:%M:%S')}"

training_args = TrainingArguments(
    output_dir=config['output_dir'],
    overwrite_output_dir=False, # If True, overwrite the content of the output directory. Use this to continue training if output_dir points to a checkpoint directory.
    learning_rate=2.5e-5, # TODO: lr decay.
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    max_grad_norm=10,
    num_train_epochs=15,
    save_strategy='steps',
    save_steps=1000,
    evaluation_strategy='steps',
    eval_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True, # Relates to metric_for_best_model.
    seed=config['seed'],
    data_seed=config['seed'],
    tf32=True, # Enable the TF32 mode, available in Ampere and newer GPU architectures.
    run_name=run_name,
    label_names=["labels"],
    optim='adamw_torch',
    report_to='wandb',
    logging_steps=250,
)

# Helper functions

In [5]:
def make_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

accuracy_metric = evaluate.load('accuracy')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')
precision_metric = evaluate.load('precision')


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1).flatten()
    refs = labels.flatten()
    return {
        'accuracy': accuracy_metric.compute(predictions=preds, references=refs)['accuracy'],
        'f1': f1_metric.compute(predictions=preds, references=refs, average='micro')['f1'],
        'recall': recall_metric.compute(predictions=preds, references=refs, average='micro')['recall'],
        'precision': precision_metric.compute(predictions=preds, references=refs, average='micro')['precision'],
    }

# Model

In [6]:
make_dir(config['output_dir'])

tokenizer = AutoTokenizer.from_pretrained(config['model_name'], add_prefix_space=True, id2label=id2label)

config_model = AutoConfig.from_pretrained(config['model_name']) 
config_model.num_labels = len(label2id)

model = AutoModelForTokenClassification.from_pretrained(config['model_name'],
                                                        config=config_model).to(config['device'])

Some weights of BigBirdForTokenClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Dataset and Dataloader

In [7]:
full_df = pd.read_pickle('../dataset.csv')
full_df.head()

,content,labels
id,,
73DC1D49FAD5,eletoral college can be a very good thing caus...,"[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
D840AC3957E5,"STUDENT_NAME\n\nADDRESS_NAME\n\nFebruary 22, 2...","[0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, ..."
753E320B186B,In my opinion as a student: I don't agree at t...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
C2ABDAC2BC2C,When it comes to at home learning and attendin...,"[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
B2DDBAAC084C,Y\n\nou can ask many different people for advi...,"[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."


In [8]:
train_df, val_df = train_test_split(full_df, test_size=config['validation_split'], random_state=config['seed'])

# Drop id column from train dataframe
train_df = train_df[['content', 'labels']]

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

print("train_df shape: ", train_df.shape)
print("val_df shape: ", val_df.shape)

train_df shape:  (13254, 2)
val_df shape:  (2340, 2)


In [9]:
class EssayDataset(Dataset):
  def __init__(self, df, tokenizer, max_len=config['max_length']):
        self.len = len(df)
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        essay_words = self.df.content[index].split() # Split essay by words before tokenizing.

        # Makes a dict with keys: input_ids, attention_mask.
        tokens = self.tokenizer(essay_words,
                                is_split_into_words=True, # Necessary to keep correspondance between words and labels contructed previously.
                                padding='max_length',
                                truncation=True,
                                max_length=self.max_len)
        """
        From Tokenizer's docs about word_ids:
            A list indicating the word corresponding to each token. Special tokens added by the tokenizer are mapped to None and other tokens
            are mapped to the index of their corresponding word (several tokens will be mapped to the same word index if they are parts of that word).

        This is needed to match the correct labels with the tokens, which may not have a 1:1 correspondence with the original words.
        """
        word_ids = tokens.word_ids()
        word_labels = None


        # Get original word label array for this essay.
        word_labels = self.df.labels[index]
        label_ids = []

        # Correct for tokenization mismatch.
        for word_idx in word_ids:
            # 'None' means that this is a special/reserved token, mark as -100 to be ignored later in training.
            if word_idx is None:
                label_ids.append(-100) # Magic number, automatically ignored by CrossEntropyLoss.
            else:
                label_ids.append(word_labels[word_idx])

        tokens['labels'] = label_ids


        word_ids2 = [w if w is not None else -1 for w in word_ids]
        tokens['word_ids'] = torch.as_tensor(word_ids2)

        return {k: torch.as_tensor(v) for k, v in tokens.items()}

  def __len__(self):
        return self.len

In [10]:
training_set = EssayDataset(df=train_df, tokenizer=tokenizer)
testing_set = EssayDataset(df=val_df, tokenizer=tokenizer)

In [11]:
training_set[0]

{'input_ids': tensor([  65, 4751, 4367,  ..., 1694, 1422,   66]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'labels': tensor([-100,    1,    2,  ...,    6,    6, -100]),
 'word_ids': tensor([ -1,   0,   1,  ..., 940, 941,  -1])}

# HuggingFace Trainer Setup

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=testing_set,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Training

In [13]:
for batch in trainer.get_eval_dataloader(testing_set):
    print(batch.keys())
    break

You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels'])


In [14]:
trainer.train()

  0%|          | 0/49710 [00:00<?, ?it/s]

{'loss': 1.163, 'learning_rate': 2.487427077046872e-05, 'epoch': 0.08}


  0%|          | 0/585 [00:00<?, ?it/s]

{'eval_loss': 0.9586074352264404, 'eval_accuracy': 0.32908779046474357, 'eval_f1': 0.32908779046474357, 'eval_recall': 0.32908779046474357, 'eval_precision': 0.32908779046474357, 'eval_runtime': 73.2124, 'eval_samples_per_second': 31.962, 'eval_steps_per_second': 7.99, 'epoch': 0.08}
{'loss': 0.8942, 'learning_rate': 2.474854154093744e-05, 'epoch': 0.15}


  0%|          | 0/585 [00:00<?, ?it/s]

...

  0%|          | 0/585 [00:00<?, ?it/s]

{'eval_loss': 1.2290393114089966, 'eval_accuracy': 0.35066356169871793, 'eval_f1': 0.35066356169871793, 'eval_recall': 0.35066356169871793, 'eval_precision': 0.35066356169871793, 'eval_runtime': 86.307, 'eval_samples_per_second': 27.113, 'eval_steps_per_second': 6.778, 'epoch': 11.92}
{'loss': 0.0684, 'learning_rate': 5.009052504526252e-06, 'epoch': 11.99}


  0%|          | 0/585 [00:00<?, ?it/s]

{'eval_loss': 1.2580863237380981, 'eval_accuracy': 0.35100494123931625, 'eval_f1': 0.35100494123931625, 'eval_recall': 0.35100494123931625, 'eval_precision': 0.35100494123931625, 'eval_runtime': 86.3311, 'eval_samples_per_second': 27.105, 'eval_steps_per_second': 6.776, 'epoch': 11.99}


KeyboardInterrupt: 

# Inferencing

In [3]:
torch.cuda.empty_cache()

In [4]:
from transformers import pipeline

arguMentor = pipeline("ner", model='/home/rafaelpiacsek/Documents/College/Courses/Semester_7/CS_410/FinalProject/ArguMentor/notebooks/model_test/checkpoint-39500')

In [5]:
test_essay = """
During a group project, have you ever asked a group member about adding or replacing something? Or, when you were studying for a math test, did you ever ask your parents or sibling about different ways to tackle a certain problem? Asking for other's opinions is especially beneficial as it allows for an individual to receive a variety of different views towards a given topic. Likewise, being diverse and asking many people for their opinions allows one to understand how most people percieve something. This is especially important as knowing multiple opinions can allow someone to take those views into account and sway themseleves to the general audience. Knowing different people's opinion can be beneficial in a variety of situations.

First and foremost, a great example about how knowing other's opinions is helpful is when someone is making the choice between smoking or refraining from smoking. A student can watch on a TV channel that smoking is bad, and can damage their internal organs. However, on another channel, the student can find advertisements about the most addicting smoking device that can release the most dopomine in the brain, all the while not severly harming people's lungs. This student will receive a variety of different views and opinions on a certain topic, which allows them to make the best educated choice or decision based on how they interpret what they saw. Similarily, a student can be told from his fellow classmates that smoking is fun, joyful, and makes them happy. However, if the student asks a local doctor, they will be informed differently. A doctor will most likely tell them that smoking, although seeming harmless at first, can lead to serious long term consequences.

If the student asks both his friends and his doctors, he is able to use his judgemental skills to determing which choice will be best for him in the long run.

Furthermore, asking for multiple opinions can benifit during competitions for a position slot, as cadidates needs to make decisions on what they need to say or do. For example, it can be helpful in situations like elections, both for the U.S. or simply in school. If a student is running for a position in office to represent his/her school, he/she can ask a widespread and diverse audience. First, asking other students is their best bet to obtaining information. Other students can inform him/her about what they want, like better water fountains, recess, or healthier food. Then, the student running can make changes to the way they run for the election, and on his/her speech, take a different approach. In addition, if the student running asks an adult, they will get to know a more realistic way the school can be improved. Since a student, even as a student officer, isn't able to make a significant change to a school, they can inform the school board about ways to make the school better. If someone is running for the president of the United States, a similar approach can be taken. First, they can ask the people, on social media or in speeches, about positive ways to reform our country. After the candidate receives the opinion of general audiences, they can campaign differently to match the view of those voting. All in all, asking for the opinion of multiple different people can set the candidate apart from others.

Many people only ask one type of audience for their opinion. Having only one opinion can lead to negative consequences, such as making the wrong choices related to health or education, as only one audience is adressed into making a decision. Therefore, asking multiple different people who have different backgrounds is essential to making the best choices in life. Conclusively, knowing multiple opinions on a certain matter can evidently lead to better results for individuals. 
"""
preds = arguMentor(test_essay)

In [9]:
segments = [int(pred['entity'].split('_')[-1]) for pred in preds]

In [14]:
from IPython.display import HTML
from math import ceil
import numpy as np

segment_colors = {
  0: 'rgba(0,0,0, 0.0)',
  1: 'rgba(206,95,20, 0.8)',
  2: 'rgba(114,174,146, 0.8)',
  3: 'rgba(251,174,28, 0.8)',
  4: 'rgba(81,53,51, 0.8)',
  5: 'rgba(43,112,133, 0.8)',
  6: 'rgba(200,109,142, 0.8)',
  7: 'rgba(243,218,179, 0.7)',
}

segment_names = {
  0: 'Unnanotated',
  1: 'Lead',
  2: 'Position',
  3: 'Evidence',
  4: 'Claim',
  5: 'Concluding Statement',
  6: 'Counterclaim',
  7: 'Rebuttal'
}

def generate_html_with_highlight(original_text, segment_types):

    # Hover stuff
    html_code = '<style>'
    html_code += '.segment-highlight:hover:before { content: attr(data-label); position: absolute; background: #111; color: #fff; padding: 4px 8px; border-radius: 4px; z-index: 2; font-size: 14px}'
    html_code += '.segment-highlight:hover:after { content: attr(data-label); position: absolute; background: #111; color: #fff; padding: 4px 8px; border-radius: 4px; z-index: 2; font-size: 14px}'
    html_code += '</style>'

    # Generate a legend showing what segment type each color represents
    html_code += '<p style="font-size: 18px; line-height: 1.6;background-color: white; color: black;"><b>Legend:</b><br/>'
    for segment_color, segment_name in zip(segment_colors.values(), segment_names.values()):
        html_code += f'<span style="background-color: {segment_color};">{segment_name}</span><br/>'
    html_code += '<br/></p>'

    html_code += '<p style="font-size: 18px; line-height: 1.6;">You can also hover on top of each segment to see their type<br/><br/></p>'

    # Highlight original text
    html_code += '<p style="font-size: 18px; line-height: 1.6; background-color: white; color: black;"><b>Segmented Essay:</b><br/>'
    current_segment_type = None
    for i, (word, segment_type) in enumerate(zip(original_text.split(), segment_types)):
        if i > 0 and segment_type != segment_types[i - 1]:
            html_code += f'</span>'

        if segment_type != current_segment_type:
            html_code += f'<span class="segment-highlight" style="background-color: {segment_colors[segment_type]};" data-label="{segment_names[segment_type]}">'
            current_segment_type = segment_type

        html_code += f'{word} '

    if current_segment_type is not None:
        html_code += '</span>'

    html_code += '</p>'

    return html_code


segments = [ceil(x/2) for x in segments]

html_code = generate_html_with_highlight(test_essay, segments)
HTML(html_code)